In [1]:
#dependencies
import re
from bs4 import BeautifulSoup 
from splinter import Browser
import requests
import pandas as pd
from selenium import webdriver 
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
import json
import time
from tqdm.notebook import tqdm 

In [25]:
geo_path = "data/state_data/geo/ppp_geo.csv" #data as of 11/21/21
ppp_geo= pd.read_csv(geo_path)
ppp_geo.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,0,0,0,0,0,9547507704,5/1/20,464,PPP,"SUMTER COATINGS, INC.",...,325 Chemicals,"3255 Paint, Coating, and Adhesive Manufacturing",3255.0,SOUTH CAROLINA DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"2410 Highway 15 South, Sumter, SC",33.853682,-80.369867,NaN
1,1,1,1,1,1,9777677704,5/1/20,464,PPP,"PLEASANT PLACES, INC.",...,999 Other,5617 Services to Buildings and Dwellings,5617.0,SOUTH CAROLINA DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"7684 Southrail Road, North Charleston, SC",32.942332,-80.066165,NaN
2,2,2,2,2,2,5791407702,5/1/20,1013,PPP,BOYER CHILDREN'S CLINIC,...,NaN,NaN,NaN,SEATTLE DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"1850 BOYER AVE E, SEATTLE, WA",47.636067,-122.302559,NaN
3,3,3,3,3,3,6223567700,5/1/20,920,PPP,KIRTLEY CONSTRUCTION INC,...,999 Other,2361 Residential Building Construction,2361.0,SANTA ANA DISTRICT OFFICE,"b $350,000-1 million",20 to 49,"1661 MARTIN RANCH RD, SAN BERNARDINO, CA",34.224719,-117.372550,NaN
4,4,4,4,4,4,9662437702,5/1/20,101,PPP,AERO BOX LLC,...,999 Other,4842 Specialized Freight Trucking,4842.0,MASSACHUSETTS DISTRICT OFFICE,"b $350,000-1 million",20 to 49,"20101 Cornillie Dr, Roseville, MI",42.521828,-82.913680,NaN


In [30]:
ppp_geonull=ppp_geo.loc[ppp_geo["Lat"].isnull()] #457467
ppp_geonull.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
369,369,369,369,369,369,5048318601,3/20/21,1084,PPS,DELTA INDUSTRIAL SERVICES INC.,...,999 Other,5511 Management of Companies and Enterprises,5511.0,ALASKA DISTRICT OFFICE,"b $350,000-1 million",10 to 19,"1229 RICHARDSON HWY, DELTA JUNCTION, AK",NaN,NaN,NaN
428,428,428,428,428,428,5085327008,4/5/20,1084,PPP,"DELTA INDUSTRIAL SERVICES, INC.",...,999 Other,5511 Management of Companies and Enterprises,5511.0,ALASKA DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"1229 Richardson Hwy, DELTA JUNCTION, AK",NaN,NaN,NaN
1865,1865,1865,1865,1865,1865,4637628408,2/6/21,1084,PPS,MCKINLEY FIRE PROTECTION LLC,...,999 Other,2382 Building Equipment Contractors,2382.0,ALASKA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"1706 Ship Ave, Anchorage, AK",NaN,NaN,NaN
2274,2274,2274,2274,2274,2274,5337318401,2/8/21,1084,PPS,THE ALASKA CENTRE FOR CREATIVE RENEWAL LLC,...,999 Other,7211 Traveler Accommodation,7211.0,ALASKA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"46877 STILLPOINT TRL, HALIBUT COVE, AK",NaN,NaN,NaN
2379,2379,2379,2379,2379,2379,9891427104,4/15/20,1084,PPP,MCKINLEY FIRE PROTECTION,...,999 Other,2382 Building Equipment Contractors,2382.0,ALASKA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"1706 SHIP AVENUE, ANCHORAGE, AK",NaN,NaN,NaN


In [31]:
ppp_geonull["BorrowerState"].value_counts()

TX    76183
NY    74112
FL    60040
MI    29806
WA    23754
CO    19741
MN    19730
WI    17354
IN    16601
MO    16350
LA    13668
OR    13456
UT     9295
IA     8295
NE     5804
MS     5502
HI     4980
NM     4485
ID     4416
RI     3792
ND     3222
MT     3144
PR     2952
OK     2839
SD     2695
SC     2335
WY     2139
NV     1829
KS     1645
GU      443
VI      290
MP       82
NC       11
PA        8
AL        8
AK        5
AZ        4
TN        3
MA        3
NJ        3
AR        2
IL        2
ME        2
OH        1
GA        1
CT        1
WV        1
Name: BorrowerState, dtype: int64

In [5]:
ppp_togeoNV = ppp_geonull.loc[ppp_geonull["BorrowerState"] == "NV"]
ppp_togeoNV #8262

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
598893,598893,598893,597579,597579,3773327405,5/8/20,944,PPP,NEVADA RESTAURANT SERVICES,3645 LOSEE RAOD,...,999 Other,7224 Drinking Places (Alcoholic Beverages),7224.0,NEVADA DISTRICT OFFICE,e $5-10 million,500 or more,"3645 LOSEE RAOD, NORTH LAS VEGAS, NV",NaN,NaN,NaN
598894,598894,598894,597580,597580,7175937006,4/7/20,944,PPP,Q&D CONSTRUCTION LLC,1050 South 21st St,...,999 Other,2362 Nonresidential Building Construction,2362.0,NEVADA DISTRICT OFFICE,e $5-10 million,250 to 499,"1050 South 21st St, SPARKS, NV",NaN,NaN,NaN
598895,598895,598895,597581,597581,7646338105,7/23/20,944,PPP,GARDEN FRESH RESTAURANTS LLC DBA SWEET TOMATOE...,2080 N Rainbow Blvd,...,999 Other,7225 Restaurants and Other Eating Places,7225.0,NEVADA DISTRICT OFFICE,e $5-10 million,500 or more,"2080 N Rainbow Blvd, LAS VEGAS, NV",NaN,NaN,NaN
598896,598896,598896,597582,597582,3465167305,4/29/20,944,PPP,DIVERSIFIED PROTECTION CORPORATION,"8020 South Rainbow Boulevard, Suite 100-530",...,999 Other,"5419 Other Professional, Scientific, and Techn...",5419.0,NEVADA DISTRICT OFFICE,e $5-10 million,500 or more,"8020 South Rainbow Boulevard, Suite 100-530, L...",NaN,NaN,NaN
598897,598897,598897,597583,597583,5910497003,4/6/20,944,PPP,"GOLDEN GATE BELL, LLC",3225-A SOUTH RAINBOW RD SUITE 102,...,999 Other,7225 Restaurants and Other Eating Places,7225.0,NEVADA DISTRICT OFFICE,e $5-10 million,500 or more,"3225-A SOUTH RAINBOW RD SUITE 102, LAS VEGAS, NV",NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
607150,607150,607150,605836,605836,9365537309,5/2/20,944,PPP,BEADO ENTERPRISES LLC,9870 W RED COACH AVE,...,999 Other,"5416 Management, Scientific, and Technical Con...",5416.0,NEVADA DISTRICT OFFICE,"a $150,000-350,000",Fewer than 5,"9870 W RED COACH AVE, LAS VEGAS, NV",NaN,NaN,NaN
607151,607151,607151,605837,605837,9428908405,2/17/21,944,PPS,JACOB PACKER LLC,8076 Satin Carnation Ln,...,999 Other,5313 Activities Related to Real Estate,5313.0,NEVADA DISTRICT OFFICE,"a $150,000-350,000",Fewer than 5,"8076 Satin Carnation Ln, Las Vegas, NV",NaN,NaN,NaN
607152,607152,607152,605838,605838,9462568501,3/12/21,944,PPS,LEGACY FIRE SERVICES LLC,600 S Rock Blvd,...,999 Other,2382 Building Equipment Contractors,2382.0,NEVADA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"600 S Rock Blvd, Sparks, NV",NaN,NaN,NaN
607153,607153,607153,605839,605839,9631538407,2/17/21,944,PPS,TAHOE FRESH CO,120 Country Club Dr Ste 24L,...,999 Other,7225 Restaurants and Other Eating Places,7225.0,NEVADA DISTRICT OFFICE,"a $150,000-350,000",50 to 99,"120 Country Club Dr Ste 24L, Incline Village, NV",NaN,NaN,NaN


In [6]:
#process to identify and code for unique addresses 
cols = ["LoanNumber","full_add"]
ppp_togeoNV_add = ppp_togeoNV[cols]
ppp_togeoNV_add

,LoanNumber,full_add
598893,3773327405,"3645 LOSEE RAOD, NORTH LAS VEGAS, NV"
598894,7175937006,"1050 South 21st St, SPARKS, NV"
598895,7646338105,"2080 N Rainbow Blvd, LAS VEGAS, NV"
598896,3465167305,"8020 South Rainbow Boulevard, Suite 100-530, L..."
598897,5910497003,"3225-A SOUTH RAINBOW RD SUITE 102, LAS VEGAS, NV"
...,...,...
607150,9365537309,"9870 W RED COACH AVE, LAS VEGAS, NV"
607151,9428908405,"8076 Satin Carnation Ln, Las Vegas, NV"
607152,9462568501,"600 S Rock Blvd, Sparks, NV"
607153,9631538407,"120 Country Club Dr Ste 24L, Incline Village, NV"


In [7]:
add = ppp_togeoNV["full_add"].astype(str).to_list()

In [8]:
def sleep():
    time.sleep(1)
    
executable_path = {'executable_path': ChromeDriverManager().install()}
driver = Browser('chrome', **executable_path, headless=False)    
url = 'https://geocoding.geo.census.gov/geocoder/geographies/onelineaddress?form'
driver.visit(url)

not_found = []
lat = []
long = []
geo_id = []

i=0
for i in tqdm(range(len(add))):
        driver.find_by_id('_address_id').clear()
        driver.find_by_id('_address_id').fill(add[i])
        driver.find_by_id('proceed').click()
        
        soup = BeautifulSoup(driver.html, 'html.parser')
        results = soup.find_all('div', id="pl_gov_census_geo_geocoder_domain_AddressResult")
        
        for result in results:
            result_text = result.text
            result_lat = re.findall('[^-](\d{2}[.]+\d+)', result_text)
            result_long = re.findall('[-]\d{3}[.]+\d+', result_text) #WEST 3-digit Longitude
            result_digits = re.findall('\d+', result_text)
            startswith = [s for s in result_digits if s.startswith('32')] #NV
            fips_elements =[element for element in startswith if len(element)>=15]
            
            try:
                #print(f"{add[i]} is {result_lat}, {result_long}, {fips_elements}")
                lat.append(result_lat[1])
                long.append(result_long[0])
                geo_id.append(fips_elements)
            except (KeyError, IndexError):
                not_found.append(i)
                lat.append(None)  
                long.append(None)
                geo_id.append(None)  
                print(f"{add[i]} is Not Found")
            except (TimeoutException):
                driver.navigate.refresh()
    
        sleep()
        i=i+1



====== WebDriver manager ======
Current google-chrome version is 99.0.4844
Get LATEST chromedriver version for 99.0.4844 google-chrome
Driver [/Users/ciaraspencer/.wdm/drivers/chromedriver/mac64/99.0.4844.51/chromedriver] found in cache


3645 LOSEE RAOD, NORTH LAS VEGAS, NV is Not Found
8020 South Rainbow Boulevard, Suite 100-530, Las Vegas, NV is Not Found
3225-A SOUTH RAINBOW RD SUITE 102, LAS VEGAS, NV is Not Found
5195 LAS VEGAS BLVD., SOUTH LAS VEGAS, NV is Not Found
3225A S Rainbow Blvd Ste 102, Las Vegas, NV is Not Found
3752 LAS VEGAS BLVD S, LAS VEGAS, NV is Not Found
102 McLeod Street, YERINGTON, NV is Not Found
3700 Barron Way, Suite 2, Reno, NV is Not Found
950 MESQUITE ROAD, MESQUITE, NV is Not Found
4141 University Center Drive, LAS VEGAS, NV is Not Found
6440 Sky Pointe Dr #140-175, LAS VEGAS, NV is Not Found
2879 SAINT ROSE PKWY # 200, HENDERSON, NV is Not Found
8906 SPANISH RIDGE AVE 202, LAS VEGAS, NV is Not Found
5990 S. Buffalo Dr., LAS VEGAS, NV is Not Found
9604 PROTOTYPE CT, RENO, NV is Not Found
PO Box 2249, STATELINE, NV is Not Found
3355 LAS VEGAS BLVD 1159, LAS VEGAS, NV is Not Found
P.O. Box 18920, RENO, NV is Not Found
1980 Festival Plaza Dr, Suite 680, Las Vegas, NV is Not Found
6720 Via A

6605 GRAND MONTECITO PKWY SUITE 200, LAS VEGAS, NV is Not Found
7195 ADVANCED WAY, LAS VEGAS, NV is Not Found
Boys & Girls Club of Southern NV, LAS VEGAS, NV is Not Found
2250 Postal Dr Ste 4, Pahrump, NV is Not Found
3355 Las Vegas Blvd S # 1159, Las Vegas, NV is Not Found
3250 W Ali Baba Ln Ste C, Las Vegas, NV is Not Found
3250 W ALI BABA LANE SUITE C, LAS VEGAS, NV is Not Found
6825 REDWOOD ST, LAS VEGAS, NV is Not Found
5655 Badura Ave Ste 180, LAS VEGAS, NV is Not Found
7955 sugar pine ct 175, RENO, NV is Not Found
7582 Las Vegas Blvd S Ste 527, Las Vegas, NV is Not Found
PO Box 487, GLENBROOK, NV is Not Found
401 North Buffalo Drive # 210, Las Vegas, NV is Not Found
2101 JONES BLVD S, LAS VEGAS, NV is Not Found
6007 DEAN MARTIN DR, LAS VEGAS, NV is Not Found
2567 Nowlin Rd #104, MINDEN, NV is Not Found
9107 W Russell Rod, Las Vegas, NV is Not Found
PILOT RD 250 E Pilot Rd,, LAS VEGAS, NV is Not Found
50 WEST LIBERTY ST SUITE 980, RENO, NV is Not Found
8780 Technology Way, Reno, 

4121 WAGON TRAIL AVE, LAS VEGAS, NV is Not Found
4121 Wagon Trail Ave, Las Vegas, NV is Not Found
3061 S MARYLAND PKWY SUITE 101, LAS VEGAS, NV is Not Found
3061 S Maryland Pkwy Ste 101, Las Vegas, NV is Not Found
8924 SPANISH RIDGE AVE, LAS VEGAS, NV is Not Found
8912 Spanish Ridge Ave Ste 200, Las Vegas, NV is Not Found
6592 DECATUR BLVD N, LAS VEGAS, NV is Not Found
6593 South Las Vegas Boulevard L, B214, LAS VEGAS, NV is Not Found
681 SIERRA ROSE DR STE. B, RENO, NV is Not Found
6630 Arroyo Springs #900, LAS VEGAS, NV is Not Found
2855 US HWY 40 WEST, VERDI, NV is Not Found
3930 Las Vegas Blvd S Ste 120, Las Vegas, NV is Not Found
240 TELEGRAPH ST, RENO, NV is Not Found
PO BOX 475, VERDI, NV is Not Found
4660 BERG ST STE 130, NORTH LAS VEGAS, NV is Not Found
4660 Berg St Ste 130, North Las Vegas, NV is Not Found
4445 VALLEY BLVD S, LAS VEGAS, NV is Not Found
10975 OVAL PARK DR, LAS VEGAS, NV is Not Found
6445 stenaya way suite 150, LAS VEGAS, NV is Not Found
3520 OQUENDO RD W, LAS 

701 EAST BRIDER AVENUE SUITE 600, LAS VEGAS, NV is Not Found
850 LAS BLVD N, LAS VEGAS, NV is Not Found
16580 WEDGE PKWY STE 300, RENO, NV is Not Found
7241 West Charleston Blvd 120, LAS VEGAS, NV is Not Found
PO BOX 475, VERDI, NV is Not Found
THE FORUM SHOPS 3500 LAS VEGAS BLVD S, LAS VEGAS, NV is Not Found
4710 West Post Road 145, Las Vegas, NV is Not Found
5470 LOUIE LN STE 104, RENO, NV is Not Found
6950 SMOKE RANCH RD STE 200, LAS VEGAS, NV is Not Found
3025 S. Sammy Davis Junior Dr., Las Vegas, NV is Not Found
8923 FLYING AVE, LAS VEGAS, NV is Not Found
3930 LAS VEGAS BLVD S STE 130, LAS VEGAS, NV is Not Found
1701 Wellness Way Ste 200, Las Vegas, NV is Not Found
330 S Rampart Suite 380, LAS VEGAS, NV is Not Found
880 SAHARA AVE E, LAS VEGAS, NV is Not Found
800 Bennie Ln Ste A, Reno, NV is Not Found
PO Box 2067, MESQUITE, NV is Not Found
235 WARM SPRINGS RD STE 107, LAS VEGAS, NV is Not Found
7260, LAS VEGAS, NV is Not Found
197 E CALIFORNIA ST #300, LAS VEGAS, NV is Not Found


...

18124 WEDGE PKWY 1008, RENO, NV is Not Found
3685 South Highland Drive 14, Las Vegas, NV is Not Found
4000 EAST CHARLESTON BLVD SUITE 100, LAS VEGAS, NV is Not Found
6671 Las Vegas Blvd S Unit 210, Las Vegas, NV is Not Found
10955 OVAL PARK DR, LAS VEGAS, NV is Not Found
P.O. Box 5611, FALLON, NV is Not Found
8195 VIRGINIA ST S, RENO, NV is Not Found
6725 Via Austi Pkwy Ste 380, Las Vegas, NV is Not Found
6725 VIA AUSTI PKWY SUITE 300, LAS VEGAS, NV is Not Found
2850 S JONES BLVD #2, LAS VEGAS, NV 89146, LAS VEGAS, NV is Not Found
2625 Green Valley Parkway Ste 270, HENDERSON, NV is Not Found
505 USA PKWY, SPARKS, NV is Not Found
31 Highway 50, Stateline, NV is Not Found
2560 Anthem Village Dr Ste 140, Henderson, NV is Not Found
2300 WEST SAHARA AVE. SUITE 950, LAS VEGAS, NV is Not Found
3157 N RAINBOW 518, LAS VEGAS, NV is Not Found
3109 PO BOX, 290 Kingsbury Grade #3, STATELINE, NV is Not Found
9490 South Eastern Ave., Suite 100, LAS VEGAS, NV is Not Found
8640 Canyon River Dive, SPAR

4616 PO BOX, CARSON CITY, NV is Not Found
7120 RAFAEL RIDGE WAY, LAS VEGAS, NV is Not Found
880 seven hills dr Suite 180, Henderson, NV is Not Found
311 WARM RD E, LAS VEGAS, NV is Not Found
6611 south las vegas boulevard Suite A 160, LAS VEGAS, NV is Not Found
10845 Griffith Peak Dr Ste 520, Las Vegas, NV is Not Found
8350 SAHARA AVE W, LAS VEGAS, NV is Not Found
1024 Normandy Ln, Mesquite, NV is Not Found
8886 Spanish Ridge Ave, Las Vegas, NV is Not Found
1600 Raiders Way Ste 150, Henderson, NV is Not Found
2970 SAINT ROSE PKWY STE 120, HENDERSON, NV is Not Found
635 SIERRA ROSE DR Suite A, RENO, NV is Not Found
6855 South Virginia Street 6855 South Virginia Street, Reno, NV is Not Found
740 Bennie Ln B, Reno, NV is Not Found
10 RED RD, MOUND HOUSE, NV is Not Found
500 MONROE AVE W, LAS VEGAS, NV is Not Found
280 GREG ST STE 2, RENO, NV is Not Found
OQUENDO RD 5880 W Oquendo Rd, LAS VEGAS, NV is Not Found
2505 Anthem Village Drive,Suite E283, Henderson, NV is Not Found
235 London Dr,

1139 US Hwy 93 Box 860, Panaca, NV is Not Found
860 South Rancho Drive #6, Las Vegas, NV is Not Found
50 West Liberty Street Suite 330, RENO, NV is Not Found
195 HWY 50 STE 104, STATELINE, NV is Not Found
6980 SMOKE RANCH RD Suite 110, LAS VEGAS, NV is Not Found
1820 WHITNEY DR 100, HENDERSON, NV is Not Found
2675 South Jones Blvd Ste 102, LAS VEGAS, NV is Not Found
2775 HIGHWAY  W, VERDI, NV is Not Found
5160 VALLEY BLVD S, LAS VEGAS, NV is Not Found
6949 SPEEDWAY BLVD. #101, LAS VEGAS, NV is Not Found
6775 Speedway Blvd, Las Vegas, NV is Not Found
8520 West Desert Inn #1, Las Vegas, NV is Not Found
2920 S. Jones #100, LAS VEGAS, NV is Not Found
40 MOJAVE RD N, LAS VEGAS, NV is Not Found
5440 LOUIE LN STE 106, RENO, NV is Not Found
3663 LAS VEGAS BLVD S STE 560, LAS VEGAS, NV is Not Found
212 Elks Point Rd Unit 101, Zephyr Cove, NV is Not Found
8485 W. S ROAD SUITE 200, LAS VEGAS, NV is Not Found
8600 WEST CHARLESTON BLVD UNIT 2049, LAS VEGAS, NV is Not Found
1920 Farm District Rd, Fe

320 CONEY ISLAND DR, SPARKS, NV is Not Found
6955 N DURANGO DR $115-285, LAS VEGAS, NV is Not Found
5626 RAFAEL RIVERA WAY, LAS VEGAS, NV is Not Found
2125 Green Vista Dr Suite 104, Sparks, NV is Not Found
6611 Las Vegas Blvd S Unit 160, Las Vegas, NV is Not Found
6055 Surrey St Ste 102, Las Vegas, NV is Not Found
840 South Rancho Drive Suite 4-569, LAS VEGAS, NV is Not Found
12 OGDEN AVE E, LAS VEGAS, NV is Not Found
7293 W. SAHARA #4, LAS VEGAS, NV is Not Found
3037 WARM RD E 100, LAS VEGAS, NV is Not Found
945 SPICE DR B, SPARKS, NV is Not Found
8461 FARM RD, STE 120-120, LAS VEGAS, NV is Not Found
501 SOUTH RANCHO DRIVE SUITE E-30, LAS VEGAS, NV is Not Found
4780 W ANN RD #5-237, NORTH LAS VEGAS, NV is Not Found
6767 W Tropicana Suite 101, Las Vegas, NV is Not Found
PO BOX 90790, HENDERSON, NV is Not Found
10760 GRASS RD S, WINNEMUCCA, NV is Not Found
1203 east Winnemucca  BNVD, Winnemucca, NV is Not Found
4030 SOUTH JONES BLVD # 30206, LAS VEGAS, NV is Not Found
STE 306 9280 W SUN

3753 HOWARD PKWY, LAS VEGAS, NV is Not Found
2520 ST ROSE PKWY 214, HENDERSON, NV is Not Found
5270 S Ft Apache Rd 330, las vegas, NV is Not Found
201 GIBSON RD S, HENDERSON, NV is Not Found
7285 Dean Martin Dr Ste 180, Las Vegas, NV is Not Found
6195 Stone Rise Street, LAS VEGAS, NV is Not Found
3852 Palos Verdes, Las Vegas, NV is Not Found
2265 VILLAGE WALK DR SUITE 105, HENDERSON, NV is Not Found
4530 ANDREW ST. #H, NORTH LAS VEGAS, NV is Not Found
2560, LAS VEGAS, NV is Not Found
951 Jacks Valley Road, Carson City, NV is Not Found
2650 Lake Sahara Dr Ste 190, Las Vegas, NV is Not Found
5020 ALTA DR STE B, LAS VEGAS, NV is Not Found
0 PO BOX 17983, RENO, NV is Not Found
5086 Post Road Building #4, Las Vegas, NV is Not Found
6293 Dean Martin Dr, Las Vegas, NV is Not Found
1455 Deming Way #13 & 14, Sparks, NV is Not Found
860 SEVEN DR, HENDERSON, NV is Not Found
5803 West Craig Rd #105, LAS VEGAS, NV is Not Found
7930 SUGAR PINE CT Suite B, RENO, NV is Not Found
112 north reservation 

In [9]:
print(len(add))
print(len(lat))
print(len(long))
print(len(geo_id))

8262
8262
8262
8262


In [ ]:
#pd.set_option('display.max_rows', None)
lat_df = pd.DataFrame({"Lat":lat})
lat_df

<b>Add remaining values (2801) to result_df(5461)

In [10]:
result_df = pd.DataFrame({"full_add":add, "Lat":lat, "Long":long, "FIPS_z": geo_id})
result_df 

,full_add,Lat,Long,FIPS_z
0,"3645 LOSEE RAOD, NORTH LAS VEGAS, NV",None,None,None
1,"1050 South 21st St, SPARKS, NV",39.524918,-119.77139,[320310031112040]
2,"2080 N Rainbow Blvd, LAS VEGAS, NV",36.19727,-115.24197,[320030034191004]
3,"8020 South Rainbow Boulevard, Suite 100-530, L...",None,None,None
4,"3225-A SOUTH RAINBOW RD SUITE 102, LAS VEGAS, NV",None,None,None
...,...,...,...,...
8257,"9870 W RED COACH AVE, LAS VEGAS, NV",36.244274,-115.31219,[320030032401006]
8258,"8076 Satin Carnation Ln, Las Vegas, NV",36.308456,-115.3001,[320030032292002]
8259,"600 S Rock Blvd, Sparks, NV",39.53895,-119.76541,[320310019032009]
8260,"120 Country Club Dr Ste 24L, Incline Village, NV",39.24043,-119.94125,"[320310033072007, 320310033072007]"


In [11]:
result_df['FIPS_z'] = result_df['FIPS_z'].str[0]
result_df 

,full_add,Lat,Long,FIPS_z
0,"3645 LOSEE RAOD, NORTH LAS VEGAS, NV",None,None,None
1,"1050 South 21st St, SPARKS, NV",39.524918,-119.77139,320310031112040
2,"2080 N Rainbow Blvd, LAS VEGAS, NV",36.19727,-115.24197,320030034191004
3,"8020 South Rainbow Boulevard, Suite 100-530, L...",None,None,None
4,"3225-A SOUTH RAINBOW RD SUITE 102, LAS VEGAS, NV",None,None,None
...,...,...,...,...
8257,"9870 W RED COACH AVE, LAS VEGAS, NV",36.244274,-115.31219,320030032401006
8258,"8076 Satin Carnation Ln, Las Vegas, NV",36.308456,-115.3001,320030032292002
8259,"600 S Rock Blvd, Sparks, NV",39.53895,-119.76541,320310019032009
8260,"120 Country Club Dr Ste 24L, Incline Village, NV",39.24043,-119.94125,320310033072007


In [12]:
ppp_togeoNV_merge = ppp_togeoNV.merge(result_df, on="full_add", how ="left")
ppp_togeoNV_merge["Lat"] = ppp_togeoNV_merge["Lat_y"]
ppp_togeoNV_merge["Long"] = ppp_togeoNV_merge["Long_y"]
ppp_togeoNV_merge["FIPS_z"] = ppp_togeoNV_merge["FIPS_z_y"]
ppp_togeoNV_merge = ppp_togeoNV_merge.drop(["Lat_x", "Long_x", "Lat_y", "Long_y", "FIPS_z_x", "FIPS_z_y"], axis=1)
ppp_togeoNV_merge


,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,598893,598893,597579,597579,3773327405,5/8/20,944,PPP,NEVADA RESTAURANT SERVICES,3645 LOSEE RAOD,...,999 Other,7224 Drinking Places (Alcoholic Beverages),7224.0,NEVADA DISTRICT OFFICE,e $5-10 million,500 or more,"3645 LOSEE RAOD, NORTH LAS VEGAS, NV",None,None,None
1,598894,598894,597580,597580,7175937006,4/7/20,944,PPP,Q&D CONSTRUCTION LLC,1050 South 21st St,...,999 Other,2362 Nonresidential Building Construction,2362.0,NEVADA DISTRICT OFFICE,e $5-10 million,250 to 499,"1050 South 21st St, SPARKS, NV",39.524918,-119.77139,320310031112040
2,598895,598895,597581,597581,7646338105,7/23/20,944,PPP,GARDEN FRESH RESTAURANTS LLC DBA SWEET TOMATOE...,2080 N Rainbow Blvd,...,999 Other,7225 Restaurants and Other Eating Places,7225.0,NEVADA DISTRICT OFFICE,e $5-10 million,500 or more,"2080 N Rainbow Blvd, LAS VEGAS, NV",36.19727,-115.24197,320030034191004
3,598896,598896,597582,597582,3465167305,4/29/20,944,PPP,DIVERSIFIED PROTECTION CORPORATION,"8020 South Rainbow Boulevard, Suite 100-530",...,999 Other,"5419 Other Professional, Scientific, and Techn...",5419.0,NEVADA DISTRICT OFFICE,e $5-10 million,500 or more,"8020 South Rainbow Boulevard, Suite 100-530, L...",None,None,None
4,598897,598897,597583,597583,5910497003,4/6/20,944,PPP,"GOLDEN GATE BELL, LLC",3225-A SOUTH RAINBOW RD SUITE 102,...,999 Other,7225 Restaurants and Other Eating Places,7225.0,NEVADA DISTRICT OFFICE,e $5-10 million,500 or more,"3225-A SOUTH RAINBOW RD SUITE 102, LAS VEGAS, NV",None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10135,607150,607150,605836,605836,9365537309,5/2/20,944,PPP,BEADO ENTERPRISES LLC,9870 W RED COACH AVE,...,999 Other,"5416 Management, Scientific, and Technical Con...",5416.0,NEVADA DISTRICT OFFICE,"a $150,000-350,000",Fewer than 5,"9870 W RED COACH AVE, LAS VEGAS, NV",36.244274,-115.31219,320030032401006
10136,607151,607151,605837,605837,9428908405,2/17/21,944,PPS,JACOB PACKER LLC,8076 Satin Carnation Ln,...,999 Other,5313 Activities Related to Real Estate,5313.0,NEVADA DISTRICT OFFICE,"a $150,000-350,000",Fewer than 5,"8076 Satin Carnation Ln, Las Vegas, NV",36.308456,-115.3001,320030032292002
10137,607152,607152,605838,605838,9462568501,3/12/21,944,PPS,LEGACY FIRE SERVICES LLC,600 S Rock Blvd,...,999 Other,2382 Building Equipment Contractors,2382.0,NEVADA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"600 S Rock Blvd, Sparks, NV",39.53895,-119.76541,320310019032009
10138,607153,607153,605839,605839,9631538407,2/17/21,944,PPS,TAHOE FRESH CO,120 Country Club Dr Ste 24L,...,999 Other,7225 Restaurants and Other Eating Places,7225.0,NEVADA DISTRICT OFFICE,"a $150,000-350,000",50 to 99,"120 Country Club Dr Ste 24L, Incline Village, NV",39.24043,-119.94125,320310033072007


In [13]:
cols = ["LoanNumber", "FIPS_z", "Lat", "Long"]
ppp_togeoNV_parsed = ppp_togeoNV_merge[cols]
ppp_togeoNV_parsed.head()

,LoanNumber,FIPS_z,Lat,Long
0,3773327405,None,None,None
1,7175937006,320310031112040,39.524918,-119.77139
2,7646338105,320030034191004,36.19727,-115.24197
3,3465167305,None,None,None
4,5910497003,None,None,None


In [14]:
ppp_geo = ppp_geo.merge(ppp_togeoNV_parsed, on="LoanNumber", how="left")
ppp_geo.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,...,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat_x,Long_x,FIPS_z_x,FIPS_z_y,Lat_y,Long_y
0,0,0,0,0,9547507704,5/1/20,464,PPP,"SUMTER COATINGS, INC.",2410 Highway 15 South,...,SOUTH CAROLINA DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"2410 Highway 15 South, Sumter, SC",33.853682,-80.369867,NaN,NaN,NaN,NaN
1,1,1,1,1,9777677704,5/1/20,464,PPP,"PLEASANT PLACES, INC.",7684 Southrail Road,...,SOUTH CAROLINA DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"7684 Southrail Road, North Charleston, SC",32.942332,-80.066165,NaN,NaN,NaN,NaN
2,2,2,2,2,5791407702,5/1/20,1013,PPP,BOYER CHILDREN'S CLINIC,1850 BOYER AVE E,...,SEATTLE DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"1850 BOYER AVE E, SEATTLE, WA",47.636067,-122.302559,NaN,NaN,NaN,NaN
3,3,3,3,3,6223567700,5/1/20,920,PPP,KIRTLEY CONSTRUCTION INC,1661 MARTIN RANCH RD,...,SANTA ANA DISTRICT OFFICE,"b $350,000-1 million",20 to 49,"1661 MARTIN RANCH RD, SAN BERNARDINO, CA",34.224719,-117.372550,NaN,NaN,NaN,NaN
4,4,4,4,4,9662437702,5/1/20,101,PPP,AERO BOX LLC,20101 Cornillie Dr,...,MASSACHUSETTS DISTRICT OFFICE,"b $350,000-1 million",20 to 49,"20101 Cornillie Dr, Roseville, MI",42.521828,-82.913680,NaN,NaN,NaN,NaN


In [15]:
ppp_geo["Lat"] = ppp_geo["Lat_y"].fillna(ppp_geo["Lat_x"])
ppp_geo["Long"] = ppp_geo["Long_y"].fillna(ppp_geo["Long_x"])
ppp_geo["FIPS_z"] = ppp_geo["FIPS_z_y"].fillna(ppp_geo["FIPS_z_x"])
ppp_geo = ppp_geo.drop(["Lat_x", "Long_x", "Lat_y", "Long_y", "FIPS_z_y", "FIPS_z_x"], axis=1)
ppp_geo.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,0,0,0,0,9547507704,5/1/20,464,PPP,"SUMTER COATINGS, INC.",2410 Highway 15 South,...,325 Chemicals,"3255 Paint, Coating, and Adhesive Manufacturing",3255.0,SOUTH CAROLINA DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"2410 Highway 15 South, Sumter, SC",33.8537,-80.3699,NaN
1,1,1,1,1,9777677704,5/1/20,464,PPP,"PLEASANT PLACES, INC.",7684 Southrail Road,...,999 Other,5617 Services to Buildings and Dwellings,5617.0,SOUTH CAROLINA DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"7684 Southrail Road, North Charleston, SC",32.9423,-80.0662,NaN
2,2,2,2,2,5791407702,5/1/20,1013,PPP,BOYER CHILDREN'S CLINIC,1850 BOYER AVE E,...,NaN,NaN,NaN,SEATTLE DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"1850 BOYER AVE E, SEATTLE, WA",47.6361,-122.303,NaN
3,3,3,3,3,6223567700,5/1/20,920,PPP,KIRTLEY CONSTRUCTION INC,1661 MARTIN RANCH RD,...,999 Other,2361 Residential Building Construction,2361.0,SANTA ANA DISTRICT OFFICE,"b $350,000-1 million",20 to 49,"1661 MARTIN RANCH RD, SAN BERNARDINO, CA",34.2247,-117.373,NaN
4,4,4,4,4,9662437702,5/1/20,101,PPP,AERO BOX LLC,20101 Cornillie Dr,...,999 Other,4842 Specialized Freight Trucking,4842.0,MASSACHUSETTS DISTRICT OFFICE,"b $350,000-1 million",20 to 49,"20101 Cornillie Dr, Roseville, MI",42.5218,-82.9137,NaN


In [24]:
ppp_geo.to_csv("data/state_data/geo/ppp_geo.csv")

In [21]:
null_check = ppp_togeoNV_parsed.loc[ppp_togeoNV_parsed["Lat"].isnull()]
null_check

,LoanNumber,FIPS_z,Lat,Long
0,3773327405,None,None,None
3,3465167305,None,None,None
4,5910497003,None,None,None
6,9038627206,None,None,None
10,5843777700,None,None,None
...,...,...,...,...
10115,6418008300,None,None,None
10116,7047647106,None,None,None
10117,7068327310,None,None,None
10118,7204258304,None,None,None
